##  [망고플레이트 크롤링](https://www.mangoplate.com/)    
### [참고 사이트](https://juwon2021.tistory.com/414)   

### ** 시작 전 확인 사항 **
  Window terminal에서   
- conda activate   
- conda activate ml-env   
(ml-env) 환경 확인 후   
- jupyter notebook   
- conda install -c anaconda requests   
- conda install bs4   
- conda install lxml   
- conda install -c conda-forge selenium   
- conda install tqdm   
- connda install -c conda-forge folium   
- connda install -c conda-forge googlemaps   
- pip install chromedriver-autoinstaller   

### 페이지 1개씩 크롤링 (for문 활용)

In [ ]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 지역 키워드를 입력하기
keyword = '강남'

# page 정하기
page = 1

In [ ]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.maximize_window() # 창 최대화

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [ ]:
### 현재 광고창이 없어서 skip

# 광고창 iframe 접근하기
driver.switch_to.frame("google_ads_iframe_/395211568/init/desktop_all_0")

# '다시 보지 않기' 버튼 클릭하기
element = '#ad > div > button.ad_btn.ad_block_btn'
driver.find_element(By.CSS_SELECTOR, element).click()

In [ ]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 200)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.find_element(By.CSS_SELECTOR, element).click()
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
            total_raw = driver.find_element(By.CSS_SELECTOR, element)
            total = total_raw.text
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            # 음식 종류 및 메뉴 크롤링
            element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
            type_raw = driver.find_element(By.CSS_SELECTOR, element1)
            type_raw = type_raw.text
            food_type = type_raw
            try:
                element2 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(9) > td > ul'
                menu_raw = driver.find_element(By.CSS_SELECTOR, element2)
                food_menu = menu_raw.text
                food_type = [type_raw, food_menu]
            except:
                pass
            
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['Point'] = total
            sub_dict['View'] = view
            sub_dict['Review'] = num_review
            sub_dict['Star'] = num_star
            sub_dict['Type'] = food_type
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    driver.close()
    return df

In [ ]:
df = FoodCrawling(row=10, col=2)
df